In [1]:
# Packages import
import os
current_dir = os.getcwd()
os.chdir('..')
print(f'Moving from {current_dir} to {os.getcwd()}')
import numpy as np
import yaml
import time
from source.utils.masks import *
from source.utils.misc import *

Moving from /Users/mqsirera/RESEARCH/CaP2/sandbox to /Users/mqsirera/RESEARCH/CaP2


In [2]:
# Files to load
config_path = './config/cifar100.yaml' 
partition_path = './config/resnet101-np4.yaml' 

In [3]:
def load_yaml(filepath):
    with open(filepath, 'r') as stream:
        try:
            data = yaml.load(stream, yaml.FullLoader)
        except yaml.YAMLError as exc:
            print(exc)
    return data

In [4]:
# Define config and partition files
configs = load_yaml(config_path)
configs['partition_path'] = partition_path
model = get_model_from_code(configs)

In [5]:
configs = partition_generator(configs, model)

num_partition: {'conv1.weight': 4, 'inputs': 4, 'layer1.0.conv1.weight': 4, 'layer1.0.conv2.weight': 4, 'layer1.1.conv1.weight': 4, 'layer1.1.conv2.weight': 4, 'layer1.2.conv1.weight': 4, 'layer1.2.conv2.weight': 4, 'layer2.0.conv1.weight': 4, 'layer2.0.conv2.weight': 4, 'layer2.0.shortcut.0.weight': 4, 'layer2.1.conv1.weight': 4, 'layer2.1.conv2.weight': 4, 'layer2.2.conv1.weight': 4, 'layer2.2.conv2.weight': 4, 'layer2.3.conv1.weight': 4, 'layer2.3.conv2.weight': 4, 'layer3.0.conv1.weight': 4, 'layer3.0.conv2.weight': 4, 'layer3.0.shortcut.0.weight': 4, 'layer3.1.conv1.weight': 4, 'layer3.1.conv2.weight': 4, 'layer3.10.conv1.weight': 4, 'layer3.10.conv2.weight': 4, 'layer3.11.conv1.weight': 4, 'layer3.11.conv2.weight': 4, 'layer3.12.conv1.weight': 4, 'layer3.12.conv2.weight': 4, 'layer3.13.conv1.weight': 4, 'layer3.13.conv2.weight': 4, 'layer3.14.conv1.weight': 4, 'layer3.14.conv2.weight': 4, 'layer3.15.conv1.weight': 4, 'layer3.15.conv2.weight': 4, 'layer3.16.conv1.weight': 4, 'laye

In [6]:
configs['comm_costs'] = set_communication_cost(model, configs['partition'],)

In [7]:
def compute_cost_matrix(layer_name, layer_weights, partition):
    """
    Computes the cost of processing an output neuron/channel at each partition.

    Args:
        layer_name (str): The name of the layer.
        layer_weights (torch.Tensor): The weight tensor for the layer.
                                      - For Conv2D: (out_channels, in_channels, kernel_height, kernel_width)
                                      - For Linear: (out_features, in_features)
        partition (dict): The partition information for the layer, including:
                          - 'num': Number of partitions
                          - 'filter_id': List of filter IDs for each partition
                          - 'channel_id': List of input channel IDs for each partition
                          - 'maps': Communication cost map between partitions

    Returns:
        np.ndarray: A cost matrix of shape (num_neurons, num_partitions), where:
                    - cost_matrix[i, j] = cost of computing output neuron `i` at partition `j`.
    """
    if layer_name not in partition:
        raise ValueError(f"Layer {layer_name} is not in the partition dictionary.")

    # Extract partition details
    num_partitions = partition[layer_name]['num']
    filter_ids = partition[layer_name]['filter_id']  # Output filters/channels per partition
    channel_ids = partition[layer_name]['channel_id']  # Input channels per partition
    maps = partition[layer_name]['maps']  # Communication cost between partitions

    # Get weight details
    weight_np = layer_weights.cpu().detach().numpy()  # Convert weights to NumPy
    shape = weight_np.shape

    # Initialize cost matrix
    num_neurons = shape[0]  # Number of output neurons
    cost_matrix = np.zeros((num_neurons, num_partitions))

    # Check if the layer is convolutional or fully connected
    is_convolutional = len(shape) == 4  # (out_channels, in_channels, kernel_height, kernel_width)

    # Iterate over output neurons and partitions
    for i in range(num_neurons):  # For each output neuron (or filter)
        for j in range(num_partitions):  # For each partition
            # Calculate cost for input channels from other partitions
            for k in range(num_partitions):  # Input channels from partition k
                if k != j:
                    input_indices = channel_ids[k]  # Input channels for partition k

                    if is_convolutional:
                        # For convolutional layers, check if all kernel elements are zero
                        active_weights = weight_np[i, input_indices, :, :]  # Weights for filter `i` and input channels
                        all_zero = np.all(active_weights == 0, axis=(1, 2))  # Check if all kernel elements are zero
                        num_active = np.sum(~all_zero)  # Count active input channels
                    else:
                        # For fully connected layers, check directly
                        active_weights = weight_np[i, input_indices]  # Weights for input neurons
                        all_zero = active_weights == 0  # Check if weights are zero
                        num_active = np.sum(~all_zero)  # Count active input neurons

                    if num_active > 0:
                        # Add the scaled cost based on the number of active channels/filters
                        cost_matrix[i, j] += (maps[j][k] * num_active)

    return cost_matrix

In [8]:
# Partition information
partition = {
    'conv1.weight': {
        'num': 3,  # 3 partitions
        'filter_id': [np.array([0, 1]), np.array([2, 3]), np.array([4, 5])],  # Filters per partition
        'channel_id': [np.array([0, 1]), np.array([2]), np.array([3])],  # Input channels per partition
        'maps': [[0, 1, 2], [1, 0, 1], [2, 1, 0]]  # Communication cost maps
    }
}

# Weights for conv1.weight layer (4D: Conv2D)
weights = torch.tensor([
    [[[1, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]],  # Filter 0
    [[[0, 0], [0, 0]], [[1, 0], [0, 0]], [[0, 0], [1, 0]], [[0, 0], [0, 1]]],  # Filter 1
    [[[1, 1], [1, 0]], [[0, 0], [0, 0]], [[1, 0], [0, 1]], [[0, 0], [0, 1]]],  # Filter 2
    [[[0, 0], [1, 0]], [[1, 0], [0, 0]], [[0, 1], [0, 0]], [[0, 0], [0, 0]]],  # Filter 3
    [[[0, 1], [0, 0]], [[0, 0], [1, 1]], [[1, 0], [0, 0]], [[0, 0], [0, 0]]],  # Filter 4
    [[[1, 0], [0, 0]], [[0, 1], [1, 1]], [[0, 0], [0, 0]], [[0, 0], [1, 0]]]   # Filter 5
])



In [9]:
cost_matrix = compute_cost_matrix('conv1.weight', weights, partition)
print(cost_matrix)


[[0. 1. 2.]
 [3. 2. 3.]
 [3. 2. 3.]
 [1. 2. 5.]
 [1. 2. 5.]
 [2. 3. 4.]]


In [15]:
current_time = time.time()
num_layers = 0
print(f'Start time: {current_time} s')
for name, W in model.named_parameters():
    if name in configs['partition']:
        C = compute_cost_matrix(name, W, configs['partition'])
        num_layers += 1
        print(f'Layer {name} processed in: {time.time() - current_time} s')
        print(f'C shape: {len(C)}, {len(C[0])}')
        current_time = time.time()
        
print(f'Num. layers: {num_layers}') 

Start time: 1732648294.427691 s
Layer conv1.weight processed in: 0.020582914352416992 s
C shape: 64, 4
Layer layer1.0.conv1.weight processed in: 0.01493525505065918 s
C shape: 64, 4
Layer layer1.0.conv2.weight processed in: 0.011151790618896484 s
C shape: 64, 4
Layer layer1.1.conv1.weight processed in: 0.009544134140014648 s
C shape: 64, 4
Layer layer1.1.conv2.weight processed in: 0.008366107940673828 s
C shape: 64, 4
Layer layer1.2.conv1.weight processed in: 0.0076978206634521484 s
C shape: 64, 4
Layer layer1.2.conv2.weight processed in: 0.00710606575012207 s
C shape: 64, 4
Layer layer2.0.conv1.weight processed in: 0.01325225830078125 s
C shape: 128, 4
Layer layer2.0.conv2.weight processed in: 0.014045000076293945 s
C shape: 128, 4
Layer layer2.0.shortcut.0.weight processed in: 0.011737823486328125 s
C shape: 128, 4
Layer layer2.1.conv1.weight processed in: 0.013861894607543945 s
C shape: 128, 4
Layer layer2.1.conv2.weight processed in: 0.014148950576782227 s
C shape: 128, 4
Layer lay

In [26]:
C

array([[384., 384., 384., 384.],
       [384., 384., 384., 384.],
       [384., 384., 384., 384.],
       ...,
       [384., 384., 384., 384.],
       [384., 384., 384., 384.],
       [384., 384., 384., 384.]])

In [ ]:
def matrix_to_partition(P, original_partition, previous_partition):
    """
    Translates an assignment matrix P (neurons by machines) back into a partition dictionary.

    Args:
        P (np.ndarray): A binary matrix of shape (num_neurons, num_partitions), where:
                        - P[i, j] = 1 if output neuron `i` is executed on machine `j`, 0 otherwise.
        original_partition (dict): The original partition dictionary, used to retrieve:
                                   - maps: Communication cost between partitions
        previous_partition (dict): The previous layer partition dictionary, used to retrieve:
                                   - channel_id: Input channels per partition

    Returns:
        dict: A new partition dictionary reconstructed based on P.
    """
    # Ensure P is a NumPy array
    P = np.array(P)

    # Validate input dimensions
    num_neurons, num_partitions = P.shape
    if 'num' not in original_partition or original_partition['num'] != num_partitions:
        raise ValueError("Mismatch between P's number of partitions and the original partition dictionary.")

    # Reconstruct the partition dictionary
    new_partition = {
        'num': num_partitions,
        'filter_id': [],  # Output neurons (filters) per partition
        'channel_id': previous_partition['filter_id'],  # Retain original input channels
        'maps': original_partition['maps'],  # Retain original communication cost map
    }

    # Populate filter_id for each partition
    for j in range(num_partitions):
        new_partition['filter_id'].append(np.where(P[:, j] == 1)[0])

    return new_partition


In [ ]:
P = np.array([
    [1, 0, 0],  # Neuron 0 -> Partition 0
    [1, 0, 0],  # Neuron 1 -> Partition 0
    [0, 1, 0],  # Neuron 2 -> Partition 1
    [0, 1, 0],  # Neuron 3 -> Partition 1
    [0, 0, 1],  # Neuron 4 -> Partition 2
    [0, 0, 1],  # Neuron 5 -> Partition 2
])

original_partition = {
    'num': 3,
    'filter_id': [np.array([0, 1]), np.array([2, 3]), np.array([4, 5])],
    'channel_id': [np.array([0, 1]), np.array([2]), np.array([3])],
    'maps': [[0, 1, 2], [1, 0, 1], [2, 1, 0]],
}

In [ ]:
new_partition = matrix_to_partition(P, original_partition)
print(new_partition)